# Algorithmes de LZ78 et LZSS

## 1. Huffman

Reprenez le code des tps précédents pour avoir une fonction ``huffmanSize(text)`` qui retourne la taille, en nombre de bits, de la compression du message ``text`` au moyen de la méthode de Huffman.

In [77]:
def count(L):
   D = {}
   for a in L:
       if a not in D:
           D[a] = 1
       else:
           D[a] +=1
   return [(v,k) for (k,v) in D.items()]

class Heap:
   def __init__(self, function= lambda x:x):
       self.tab = []
       self.f = function
   def father(self,i):
       if i == 0:
           return None
       return int((i-1)/2)
   def left(self,i):
       if 2*i+1 < len(self.tab):
           return 2*i+1
       return None
   def right(self,i):
       if 2*i+2 < len(self.tab):
           return 2*i+2
       return None
   def __str__(self):
       return str(self.tab)
   def add(self,x):
       self.tab.append(x)
       i = len(self.tab)-1
       while i>0 and self.f(self.tab[i]) < self.f(self.tab[self.father(i)]):
           self.tab[i], self.tab[self.father(i)] = self.tab[self.father(i)], self.tab[i]
           i = self.father(i)
   def extract(self):
       if len(self.tab) == 1:
               return self.tab.pop()
       res = self.tab[0]
       self.tab[0] = self.tab.pop()
       i = 0
       while True:
           l,r = self.left(i), self.right(i)
           vl, vr = self.f(self.tab[i]), self.f(self.tab[i])
           if l is not None:
               vl = self.f(self.tab[l])
           if r is not None:
               vr = self.f(self.tab[r])
           if self.f(self.tab[i]) <= vr and self.f(self.tab[i]) <= vl:
               return res
           if vl < vr:
               self.tab[i], self.tab[l] = self.tab[l], self.tab[i]
               i = l
           else:
               self.tab[i], self.tab[r] = self.tab[r], self.tab[i]
               i = r    

def huffmanTree(L):
   C = count(L)
   H = Heap(lambda x:x[0])
   for c in C:
       H.add(c)
   while len(H.tab)>1:
       x1 = H.extract()
       x2 = H.extract()
       A = (x1[0]+x2[0],x1,x2)
       H.add(A)
   return H.tab[0]  

def huffmanToCode(T,C=None,word=None):
   if word is None:
       word = []
   if C is None:
       C = {}
   if len(T) == 2: # leaf
       C[T[1]] = u"".join(word)
   else:
       word.append('0')
       huffmanToCode(T[1],C,word)
       word[-1] = '1'
       huffmanToCode(T[2],C,word)
       word.pop()
   return C

from math import log

def huffmanSize(text):
   K = count(text)
   T = huffmanTree(text)
   C = huffmanToCode(T)
   k = len(K)
   b = int(log(k+1,2)+1)
   return sum([len(C[l])*nb for nb,l in K])+ (b+2)*k -1

## 2. LZ78

Implantez l'algorithme de LZ78. La fonction aura pour signature ``lz78(u, size=None)``, où ``size`` représente la taille maximale du dictionnaire (voir le cours). Si ``size`` vaut ``None``, alors le dictionnaire n'a pas de limite de taille. Sinon, il est vidé dès qu'il atteint cette taille (voir le cours), et réinitialisé pour ne plus contenir que le mot vide.

La sortie de l'algorithme sera, comme dans le cours, une liste de paires (numéro,caractère).

In [78]:
def lz78(u, size=None):
    output = list()
    dictionnaire = dict()
    dictionnaire[''] = 0
    compteur = 1
    i = 0
    while i < len(u):
        if len(dictionnaire) == size:
            dictionnaire = dict()
            dictionnaire[''] = 0
        elif u[i] not in dictionnaire:
            dictionnaire[u[i]] = compteur
            output.append((0, u[i]))
            compteur +=1
            i+=1
        else:
            l = 0
            while u[i:i+l] in dictionnaire.keys() and i + l < len(u):
                l+=1
            dictionnaire[u[i:i+l]] = compteur
            output.append((dictionnaire[u[i:i+l-1]], u[i:i+l][-1]))
            compteur+=1
            i+=l
    return output

Vérifiez que votre fonction fonctionne correctement avec l'exemple du cours.

In [56]:
print(lz78("abbababaaab"))

#Resultat : (0,a) (0,b) (2,a) (3,b) (1,a) (1,b)

[(0, 'a'), (0, 'b'), (2, 'a'), (3, 'b'), (1, 'a'), (1, 'b')]


Ecrire la fonction `un_lz78(seq,size=None)` qui décompresse la séquence `seq` produite par `lz78`.

In [80]:
def un_lz78(seq, size=None):
    res = ""
    dictionnaire = dict()
    dictionnaire[0] = ''
    cpt = 1
    for u, v in seq:
        ajout = dictionnaire[u] + v
        res += ajout
        dictionnaire[cpt] = ajout
        cpt+=1
    return res

Vérifiez que votre fonction fonctionne avec le script ci-dessous.

In [79]:
from random import choice
for _ in range(10):
    u = "".join([choice(['a','b','c']) for _ in range(20)])
    print(u, un_lz78(lz78(u))==u)
    print(u, un_lz78(lz78(u,5),5)==u)

ccbcbccbaaacbccaaaac True
ccbcbccbaaacbccaaaac True
aabbbcccbbbccbabaccb True
aabbbcccbbbccbabaccb True
bbccaccaccccabaabbcb True
bbccaccaccccabaabbcb True
ccabcbccbabcabbcccbb True
ccabcbccbabcabbcccbb True
aabacccccabaacbbcbbb True
aabacccccabaacbbcbbb True
bcbaaacbcbacaacbabbb True
bcbaaacbcbacaacbabbb True
cbcabbbbaacccababccc True
cbcabbbbaacccababccc True
ccbcaaccbacbccbbbccb True
ccbcaaccbacbccbbbccb True
cbbbaacaccabcabaccbb True
cbbbaacaccabcabaccbb True
baaaacbbaccbccabaaac True
baaaacbbaccbccabaaac True


## 3. Tests LZ78 + Huffman

Appliquez lz78 au fichier `etranger.txt` (en ouvrant le fichier avec l'option 'r', pas 'rb'). Quelle est la taille de la compression du résultat de lz78 par Huffman (sans limite de taille pour l'alphabet)? Comparez-là à la taille initiale du fichier `etranger.txt`.

In [82]:
with open("etranger.txt", "r") as file:
    data = file.read()
    encodage = lz78(data)
    print(huffmanSize(encodage)/len(data)/8)

0.7783588115268366


On souhaite maintenant utiliser deux arbres de Huffman : un pour les indices, et un pour les caractères. Que deviendrait la taille du fichier compressé si on appliquait cette technique à `etranger.txt` ?

Même question si en limitant la taille du dictionnaire à s=512, s=1024, ou s=2048.

Essayez d'autres stratégies quand le dictionnaire est plein (par exemple enlever la moitié des mots selon leur indice, ou selon leur taille)

## 4. LZW

Reprendre les parties 2. et 3. avec LZW au lieu de LZ78.

True